# Classification: Dogs and Cats

In this notebook, we will be using [Keras](https://keras.io/), a very high-level neural networks API, to handle a small binary classification task.

## Imports

In [ ]:
import numpy as np # module for handling matricies

import matplotlib.pyplot as plt # module for drawing/plotting stuff
import matplotlib.image as mpimg # module for images
import PIL # another module for images
%matplotlib inline

# import the necessary neural network components
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

## Prepare Data
There was a lot of data in the train and test folders initially, deleted to save space

In practice, hundreds of thousands of data samples are need to get high (90%+) accuracy on tough problems!

In [ ]:
img_width, img_height = 250, 250

train_data_dir = 'train'
validation_data_dir = 'test'

In [ ]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

In [ ]:
train_generator.class_indices # labels used for cat and dog classes respectively

#### Set up the Model

In [ ]:
model = Sequential()
model.add(Conv2D(64,(3,3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32,(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
print(model.summary()) # How many layers? How many parameters (weights)?

## Train the Network (Skip to the "Evaluate the Network" section)
The following cells should not be run unless you have a [nice](https://medium.com/@timcamber/deep-learning-pc-build-5cffa71ad97) computer (aka w/ GPU) and/or free time.

In [ ]:
nb_epoch = 1
nb_train_samples = 22778
nb_validation_samples = 2222

In [ ]:
# Train model on dataset
model.fit_generator(generator=train_generator,
                    validation_data=validation_generator)

In [ ]:
model.save_weights('new-weights-from-1-epoch.h5')

## Evaluate the Network
The prior cells to train the network have already been run once, and the trained parameters have been saved to the following file.

We can use those to save time!

In [ ]:
model.load_weights('weights-from-1-epoch.h5')

#### Evaluate on Dogs!

In [ ]:
test_dogs_lst = [920, 921, 922, 923, 924] # image numbers (must actually be in the test/dogs folder)
test_dogs_paths = ['test/dogs/dog.{}.jpg'.format(i) for i in test_dogs_lst] # corresponding file paths
test_dogs = np.asarray([mpimg.imread(path) for path in test_dogs_paths]) # model.predict needs an array of images
dog_predictions = model.predict(test_dogs) # predict!

In [ ]:
plt.figure()
fig=plt.figure(figsize=(18, 16), dpi= 80) # make things bigger
print(" " * 50 + "Prediction For Dogs") # janky way of centering title
for i,dog in enumerate(test_dogs):
    plt.subplot(1,len(test_dogs_lst),i+1)
    plt.imshow(dog, cmap='gray', interpolation='none')
    plt.title("Class Prediction {}".format(dog_predictions[i][0]))

# TODO: Evaluate on Cats!

References:

https://gggdomi.github.io/keras-workshop/notebook.html

Recommended Further Reading:

http://neuralnetworksanddeeplearning.com/chap1.html